In [1]:
from datetime import timedelta, date
from typing import Union

import pandas as pd
from plutus.data.data_process.process_cn_future_master import ProcessCnFutureMaster
from plutus.data.data_process.process_cn_future_md import ProcessCnFutureMd

In [2]:
process_cn_future_master = ProcessCnFutureMaster()
process_cn_future_md = ProcessCnFutureMd()

# future

## master

In [3]:
# contract_info_rq
trading_date_info = process_cn_future_master.read_clickhouse(
    "commercial_cn_future_master", "trading_date_info"
)
contract_info = process_cn_future_master.read_clickhouse(
    "pretreated_cn_future_master", "contract_info_rq"
)
processed_contract_info = process_cn_future_master.process_contract_info(
    trading_date_info["trading_date"], contract_info
)
process_cn_future_master.save_clickhouse(
    processed_contract_info,
    "processed_cn_future_master",
    "contract_info_rq",
    ind=["contract"],
)

2022-06-15 01:27:29 - ck_control.py - INFO - Data inserted into processed_cn_future_master.contract_info_rq: Dataframe(685, 7)
2022-06-15 01:27:29 - database_control.py - INFO - Data inserted into clickhouse processed_cn_future_master.contract_info_rq: Dataframe(685, 7)


## md

In [4]:
# main_roll_calendar_rq
pretreated_df = process_cn_future_md.read_clickhouse(
    "pretreated_cn_future_md",
    "all_1d_rq",
    filters=[("trading_date ", " > ", "'2010-01-01 00:00:00'")],
)
pretreated_df.sort_values(by=["trading_date"], inplace=True)

processed_df = process_cn_future_md.process_main_roll_calendar(
    pretreated_df, contract_info
)
processed_df.dropna(subset=["o_m", "o_nm", "v_nm", "v_m"], inplace=True)
process_cn_future_md.save_clickhouse(
    processed_df,
    "processed_cn_future_md",
    "main_roll_calendar_rq",
    ind=["trading_date", "symbol"],
    partition=["trading_date"],
)

select * from pretreated_cn_future_md.all_1d_rq where trading_date   >  '2010-01-01 00:00:00' 


2022-06-15 01:28:40 - ck_control.py - INFO - Data inserted into processed_cn_future_md.main_roll_calendar_rq: Dataframe(16196, 6)
2022-06-15 01:28:40 - database_control.py - INFO - Data inserted into clickhouse processed_cn_future_md.main_roll_calendar_rq: Dataframe(16196, 6)
